# Applications Weighted Grading Automation

(*By: [@mahmoud-elmakki](https://github.com/mahmoud-elmakki)*)

The main objecctive of this code is to automate the process of calculating the overall weighted grade of students' responses, from the weights and grades given to each question from the graded questions.

In [ ]:
import numpy as np
import pandas as pd

import os

In [ ]:
# Note that you have to download the responses data Excel sheet from Google Drive and put it in the same folder as the code.
# You don't have to do this if you cloned the Github repo (all will be organized in the repo).
 
# TODO: Load data directly from Google Drive.

# Loading students responses data
GRD_STDN_DATA_DIR = './responses_with_graded_answers/Graded TReND Comp Neuro application form Rwanda 2024 (Responses).xlsx'
grd_stdn_df = pd.read_excel(GRD_STDN_DATA_DIR)

grd_stdn_df['Total Grade'] = None

# Just specify folder names - thje code will create the directory
OUTPUT_FOLDER_NAME = "final_output"
OUTPUT_DIR = os.path.join(os.getcwd(), OUTPUT_FOLDER_NAME)

if not os.path.exists(OUTPUT_DIR):
    os.mkdir(OUTPUT_DIR)

In [ ]:
# Use this dictionary as a reference for column names.

qs_dict = {i: column for i, column in enumerate(grd_stdn_df.columns)}
qs_dict

In [ ]:
# Used indices of the student responses DataFrame

stdn_idcs = {
    'email_idx' : 1,
    'firstname_idx' : 2,
    'lastname_idx' : 3,
    'ref' : {
        'first_ref_email_idx' : 25,
        'second_ref_email_idx' : 27
          },
    'flag_idx' : 28,
    'notes_idx' : 29,
    'first_recomm_letter_idx' : 30,
    'second_recomm_letter_idx' : 31,
    'grade_idx': 28
}

stdn_str_qs = [stdn_idcs['email_idx'], stdn_idcs['firstname_idx'], stdn_idcs['lastname_idx']]

# Carefully specify names of the columns to be processed (mostly responses for essay questions).
skills_qs = [16, 17, 18, 19]
skills_qs_ws = [1, 2, 2, 2]

# Carefully specify names of the columns to be processed (mostly responses for essay questions).
essay_qs = [20, 21, 22]
essay_qs_ws = [3, 3, 2]

grd_qs = skills_qs + essay_qs
weights = skills_qs_ws + essay_qs_ws

WEIGHTED_GRADIN = True

if not WEIGHTED_GRADIN:
    weights = [1, 1, 1, 1, 1, 1, 1]

# Number of students to be selected.
TOP_N = 20

if TOP_N < len(grd_stdn_df):
    TOP_N = len(grd_stdn_df)

In [ ]:
skills_qs_dict = {
    
    16: {
        'Have not done any computer programming': 2,
        'Learned some, edited scripts': 4,
        'Have written my own sets of code': 6,
        'Have written my own complete programs': 8,
        'I am completely comfortable writing my own programs': 10
        
    },
    
    17: {
        'Have not touched Python yet': 2,
        'Learned some, edited scripts': 4,
        'Have written my own sets of code': 6,
        'Have written my own complete programs': 8,
        'I am completely comfortable writing my own programs': 10
    },
    
    18: {
        'Have read a bit': 2,
        'Have taken an introductory course': 4,
        'Have taken several courses': 6,
        'Have taken courses and am studying it': 8,
        'Extensive experience for 4+ years': 10
    },
    
    19: {
        'Have read a bit': 2,
        'Have taken an introductory course': 4,
        'Have taken several courses': 6,
        'Have taken courses and am studying it': 8,
        'Extensive experience for 4+ years': 10
    }

}

In [ ]:
def column_names_to_indices(df, indices_dict):
    """
    Replaces column names with indices.
    """
    processed_df = df.rename(columns={column: i for i, column in enumerate(indices_dict.values())})

    return processed_df


def indices_to_column_names(df, indices_dict):
    """
    Replaces indices with column names.
    """
    processed_df = df.rename(columns={i: column for i, column in enumerate(indices_dict.values())})

    return processed_df


def calc_grade(grd_stdn_df):
    
    for row_idx in range(len(grd_stdn_df)):
        grds = []
        
        for q in skills_qs:
            grd = float(skills_qs_dict[q][grd_stdn_df.iloc[row_idx, q]])
            grds.append(grd)
            
        for q in essay_qs:
            grd = float(grd_stdn_df.iloc[row_idx, q])
            grds.append(grd)
        
        weighted_grds = [grds[i] * weights[i] for i in range(len(grds))]
        total_grd = float(round(sum(weighted_grds) / sum(weights), 2))
        grd_stdn_df.iloc[row_idx, stdn_idcs['grade_idx']] = total_grd
            
    return grd_stdn_df
 

In [ ]:
def main(grd_stdn_df):
    
    grd_stdn_df = column_names_to_indices(grd_stdn_df, qs_dict)
    grd_stdn_df_graded = calc_grade(grd_stdn_df)
    grd_stdn_df_sorted = grd_stdn_df_graded.sort_values(by = stdn_idcs['grade_idx'], ascending = False)
    
    grd_stdn_df_named = indices_to_column_names(grd_stdn_df_sorted, qs_dict)
    grd_stdn_df_named.to_excel(OUTPUT_FOLDER_NAME + "/graded_responses.xlsx")
    
    top_n_stnds = grd_stdn_df_sorted.iloc[:TOP_N, :]
    
    top_n_stnds_nammed = indices_to_column_names(grd_stdn_df_graded, qs_dict)
    top_n_stnds_nammed.to_excel(OUTPUT_FOLDER_NAME + "/top_n_graded_responses.xlsx")
    
    return grd_stdn_df_sorted, top_n_stnds

In [ ]:
grd_stdn_df_graded, top_n_stnds = main(grd_stdn_df)

In [ ]:
grd_stdn_df_graded.iloc[:, stdn_idcs['grade_idx']]

In [ ]:
top_n_stnds.iloc[:, stdn_idcs['grade_idx']]

In [ ]:
top_n_stnds